In [1]:
import spacy
import re
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', None)
from spacy.pipeline import EntityRuler 

/Users/seyi/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """


In [2]:
nlp = spacy.load("en") 
ruler = EntityRuler(nlp)
patterns = [{"label":"REL", "pattern": [{"TEXT":{"REGEX": r"(?i)\b(wife?|husband?|mother?|father?|son?|daughter?|aunt?|uncle?|in-laws?|in-law?|father in-law?|mother in-law?|grandfather?|grandmother?|family?)\b"}}]}]
ruler.add_patterns(patterns)
nlp.add_pipe(ruler)

In [3]:
def get_entities(text):
    entities = {}
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ == 'DATE' or ent.label_ == 'CARDINAL':
            x = re.search("(\d+)\s*(years?|year?|months?|month?)", ent.text, re.IGNORECASE)
            if x:
                entities[ent.text] = 'AGE'
            else:
                entities[ent.text] = ent.label_
        else:
            entities[ent.text] = ent.label_
    return entities

In [4]:
labelled_data = pd.read_csv('./english_1K_sequence_label.csv')

In [5]:
labelled_data.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'index', 'GLOBALEVENTID', 'SQLDATE',
       'MonthYear', 'Year', 'FractionDate', 'Actor1Code', 'Actor1Name',
       'Actor1CountryCode', 'Actor1KnownGroupCode', 'Actor1EthnicCode',
       'Actor1Religion1Code', 'Actor1Religion2Code', 'Actor1Type1Code',
       'Actor1Type2Code', 'Actor1Type3Code', 'Actor2Code', 'Actor2Name',
       'Actor2CountryCode', 'Actor2KnownGroupCode', 'Actor2EthnicCode',
       'Actor2Religion1Code', 'Actor2Religion2Code', 'Actor2Type1Code',
       'Actor2Type2Code', 'Actor2Type3Code', 'IsRootEvent', 'EventCode',
       'CAMEOCodeDescription', 'EventBaseCode', 'EventRootCode', 'QuadClass',
       'GoldsteinScale', 'NumMentions', 'NumSources', 'NumArticles', 'AvgTone',
       'Actor1Geo_Type', 'Actor1Geo_FullName', 'Actor1Geo_CountryCode',
       'Actor1Geo_ADM1Code', 'Actor1Geo_ADM2Code', 'Actor1Geo_Lat',
       'Actor1Geo_Long', 'Actor1Geo_FeatureID', 'Actor2Geo_Type',
       'Actor2Geo_FullName', 'Actor2Geo_CountryCode', 'A

In [6]:
labelled_data['NER'] = labelled_data['summary'].apply(get_entities)

In [7]:
labelled_data1 = labelled_data[labelled_data['lab_final'] == 'DV']

In [8]:
labelled_data1.head()

Unnamed: 0  Unnamed: 0.1   index  GLOBALEVENTID   SQLDATE  MonthYear  \
7   21          19            50012   896251780      19200102  192001      
10  36          35            59131   896436364      19200103  192001      
20  82          82            74428   897031109      20200106  202001      
63  202         200           175262  897670166      20200108  202001      
82  262         258           42459   898295873      20200111  202001      

    Year  FractionDate Actor1Code Actor1Name Actor1CountryCode  \
7   1920  1920.0055     COP        POLICE     NaN                
10  1920  1920.0082     NaN        NaN        NaN                
20  2020  2020.0164     COP        POLICE     NaN                
63  2020  2020.0219     NaN        NaN        NaN                
82  2020  2020.0301     IND        JAIPUR     IND                

   Actor1KnownGroupCode Actor1EthnicCode Actor1Religion1Code  \
7   NaN                  NaN              NaN                  
10  NaN                  NaN              NaN                  
20  NaN                  NaN              NaN                  
63  NaN                  NaN              NaN                  
82  NaN                  NaN              NaN                  

   Actor1Religion2Code Actor1Type1Code Actor1Type2Code  Actor1Type3Code  \
7   NaN                 COP             NaN            NaN                
10  NaN                 NaN             NaN            NaN                
20  NaN                 COP             NaN            NaN                
63  NaN                 NaN             NaN            NaN                
82  NaN                 NaN             NaN            NaN                

   Actor2Code Actor2Name Actor2CountryCode Actor2KnownGroupCode  \
7   NaN        NaN        NaN               NaN                   
10  HLH        HOSPITAL   NaN               NaN                   
20  NaN        NaN        NaN               NaN                   
63  COP        POLICE     NaN               NaN                   
82  NaN        NaN        NaN               NaN                   

   Actor2EthnicCode Actor2Religion1Code  Actor2Religion2Code Actor2Type1Code  \
7   NaN              NaN                NaN                   NaN              
10  NaN              NaN                NaN                   HLH              
20  NaN              NaN                NaN                   NaN              
63  NaN              NaN                NaN                   COP              
82  NaN              NaN                NaN                   NaN              

   Actor2Type2Code  Actor2Type3Code  IsRootEvent  EventCode  \
7   NaN            NaN               0            114         
10  NaN            NaN               1            42          
20  NaN            NaN               0            180         
63  NaN            NaN               1            173         
82  NaN            NaN               1            190         

                                     CAMEOCodeDescription  EventBaseCode  \
7    Complain officially                                   114             
10   Make a visit                                          42              
20   Use unconventional violence, not specified below      180             
63   Arrest, detain, or charge with legal action           173             
82   Use conventional military force, not specified below  190             

    EventRootCode  QuadClass  GoldsteinScale  NumMentions  NumSources  \
7   11             3         -2.0             10           1            
10  4              1          1.9             10           1            
20  18             4         -9.0             10           1            
63  17             4         -5.0             10           1            
82  19             4         -10.0            4            1            

    NumArticles    AvgTone  Actor1Geo_Type                Actor1Geo_FullName  \
7   10          -2.626263   4               Tirupati, Andhra Pradesh, India

In [9]:
labelled_data.to_csv('./english_1K_with_NER_tags.csv')